In [2]:
import re
import pandas as pd
import numpy as np
import regex

### Python code to extract Date from chat file 

In [3]:
def startsWithDateAndTime(s):
    # pattern = '^([0-9]+)(/)([0-9]+)(/)([0-9][0-9]), ([0-9]+):([0-9][0-9]) (AM|PM) -'
    #pattern = '[0-9]{2}/[0-9]{2}/[0-9]{4}'
    pattern = '[0-9]{1}/[0-9]{2}/[0-9]{2}' # for New Group settings
    result = re.match(pattern, s)
    #print(result)
    if result:
        return True
    return False

### Regex pattern to extract username of Author.

In [4]:

def FindAuthor(s):
    # ~~~~ Produce error ~~~~~ #
    patterns = [
        '([w]+):',                        # First Name
        '([w]+[s]+[w]+):',              # First Name + Last Name
        '([w]+[s]+[w]+[s]+[w]+):',    # First Name + Middle Name + Last Name
        '([+]d{2} d{5} d{5}):',         # Mobile Number (India no.)
        '([+]d{2} d{3} d{3} d{4}):',   # Mobile Number (US no.)
        '([w]+)[u263a-U0001f999]+:',    # Name and Emoji              
    ]
    pattern = '^' + '|'.join(patterns)
    
    result = re.match(pattern, s)
    if result:
        return True
    return False

### Extracting Date, Time, Author and message from the chat file.

In [5]:

def getDataPoint(line):   
    splitLine = line.split(' - ') 
    dateTime = splitLine[0]
    date, time = dateTime.split(', ') 
    message = ' '.join(splitLine[1:])
    #if FindAuthor(message): 
    splitMessage = message.split(': ') 
    author = splitMessage[0] 
    message = ' '.join(splitMessage[1:])
    #else:
    #    author = None
    return date, time, author, message


In [6]:
### Finally creating a dataframe and storing all data inside that dataframe.
parsedData = [] # List to keep track of data so it can be used by a Pandas dataframe
### Uploading exported chat file
conversationPath = 'WhatsApp_Chat.txt' # chat file
with open(conversationPath, encoding="utf-8") as fp:
    ### Skipping first line of the file because contains information related to something about end-to-end encryption
    fp.readline()
    
    #print(fp.readline())
    #fp.readline()
    messageBuffer = [] 
    date, time, author = None, None, None
    while True:
        line = fp.readline()
        #print(line)
        if not line: 
            break
        line = line.strip() 
        if startsWithDateAndTime(line): 
            #print(100)
            if len(messageBuffer) > 0:
                #print('!!!!!!!!!!!!!!!!!')
                parsedData.append([date, time, author, ' '.join(messageBuffer)]) 
            messageBuffer.clear() 
            date, time, author, message = getDataPoint(line) 
            messageBuffer.append(message) 
        else:
            #print(200)
            messageBuffer.append(line)
    
print(len(parsedData))

df = pd.DataFrame(parsedData, columns=['Date', 'Time', 'Author', 'Message']) # Initialising a pandas Dataframe.
### changing datatype of "Date" column.
df["Date"] = pd.to_datetime(df["Date"])

29957


In [7]:
df

,Date,Time,Author,Message
0,2021-05-11,8:33 PM,Ayesha PUCIT,Hmme na pagal ho chuki har teacher ki mintaien...
1,2021-05-11,8:33 PM,Celal PUCIT,Karain gey 👍
2,2021-05-11,8:33 PM,Ayesha PUCIT,"Kbhe sessional, kbhe assignment kbhe presenta..."
3,2021-05-11,8:33 PM,Celal PUCIT,Jitni marzi karwa lo. Bs quiz ya Viva nhi dena
4,2021-05-11,8:33 PM,Ayesha PUCIT,Or phir teachers k krny waly kam b me he kron😶
...,...,...,...,...
29952,2021-08-17,4:08 PM,Shakir Jamil,I don't need that type of aurat jo MSCS ki wja...
29953,2021-08-17,4:09 PM,Adnan Ashraf PUCIT,😂😂😂 Fir to chutti kr
29954,2021-08-17,4:09 PM,Shakir Jamil,Aho chutti hi ki hoi hai😂😂😂
29955,2021-08-17,4:09 PM,Adnan Ashraf PUCIT,😂😂😂


In [8]:
### Checking shape of dataset.
df.shape

(29957, 4)

In [63]:
### Checking basic information of dataset
#df.info()

In [11]:
### Droping Nan values from dataset
df = df.dropna()
df = df.reset_index(drop=True)

'''Don't drop, just take the rows where EPS is not NA'''

#df = df[~(df['Message'].str.len == 0)]
df = df.replace(r'^\s*$', np.nan , regex=True)
df = df.replace(r'<Media omitted>$', np.nan , regex=True)
df.shape

df = df[df['Message'].notna()]

In [12]:
### Checking no. of authors of group
df['Author'].nunique()

34

In [16]:
### Checking authors of group
df['Author'].unique()

array(['Ayesha PUCIT', 'Celal PUCIT', 'F.Bilal Uol', 'Umar Ghaffar PUCIT',
       'Amna Ali PUCIT', 'Ayesha Kiran PUCIT', 'Abdullah PUCIT',
       '+92 323 4446586', 'Khurram PUCIT', '+92 322 4155225',
       'Saad PUCIT', 'Adnan Ashraf PUCIT', 'Iqra Aslam PUCIT',
       'Shakir Jamil', 'Muflah PUCIT', 'Arshia CH PUCIT',
       'Usman CH PUCIT', '+92 304 5307578', '+92 300 4611646',
       'Muhammad Faraz PUCIT', '+92 306 2628299', 'Hafsa PUCIT',
       '+92 321 7222267', '+92 307 3900838', 'Aimen PUCIT',
       '+92 303 5018620', 'Nabeel Jutt', '+92 322 6678540',
       'Saad PUCIT MSV', '+92 335 0607000', '+92 335 5027377',
       '+92 306 4093886', 'Aneesa PUCIT', '+92 303 9326488'], dtype=object)

### Adding one more column of "Day" for better analysis, here we use datetime library which help us to do this task easily.

In [17]:
weeks = {
0 : 'Monday',
1 : 'Tuesday',
2 : 'Wednesday',
3 : 'Thrusday',
4 : 'Friday',
5 : 'Saturday',
6 : 'Sunday'
}
df['Day'] = df['Date'].dt.weekday.map(weeks)
### Rearranging the columns for better understanding
df = df[['Date','Day','Time','Author','Message']]
### Changing the datatype of column "Day".
df['Day'] = df['Day'].astype('category')
### Looking newborn dataset.
#df.head()
#df.tail()

### Counting number of letters in each message
df['Letter\'s'] = df['Message'].apply(lambda s : len(s))
### Counting number of word's in each message
df['Word\'s'] = df['Message'].apply(lambda s : len(s.split(' ')))
### Function to count number of links in dataset, it will add extra column and store information in it.
URLPATTERN = r'(https?://S+)'
df['Url_Count'] = df.Message.apply(lambda x: re.findall(URLPATTERN, x)).str.len()
links = np.sum(df.Url_Count)
### Function to count number of media in chat.
MEDIAPATTERN = r'<Media omitted>'
df['Media_Count'] = df.Message.apply(lambda x : re.findall(MEDIAPATTERN, x)).str.len()
media = np.sum(df.Media_Count)

### Looking updated dataset
df

,Date,Day,Time,Author,Message,Letter's,Word's,Url_Count,Media_Count
0,2021-05-11,Tuesday,8:33 PM,Ayesha PUCIT,Hmme na pagal ho chuki har teacher ki mintaien...,57,12,0,0
1,2021-05-11,Tuesday,8:33 PM,Celal PUCIT,Karain gey 👍,12,3,0,0
2,2021-05-11,Tuesday,8:33 PM,Ayesha PUCIT,"Kbhe sessional, kbhe assignment kbhe presenta...",51,7,0,0
3,2021-05-11,Tuesday,8:33 PM,Celal PUCIT,Jitni marzi karwa lo. Bs quiz ya Viva nhi dena,46,10,0,0
4,2021-05-11,Tuesday,8:33 PM,Ayesha PUCIT,Or phir teachers k krny waly kam b me he kron😶,46,11,0,0
...,...,...,...,...,...,...,...,...,...
29952,2021-08-17,Tuesday,4:08 PM,Shakir Jamil,I don't need that type of aurat jo MSCS ki wja...,77,17,0,0
29953,2021-08-17,Tuesday,4:09 PM,Adnan Ashraf PUCIT,😂😂😂 Fir to chutti kr,20,5,0,0
29954,2021-08-17,Tuesday,4:09 PM,Shakir Jamil,Aho chutti hi ki hoi hai😂😂😂,27,6,0,0
29955,2021-08-17,Tuesday,4:09 PM,Adnan Ashraf PUCIT,😂😂😂,3,1,0,0


In [18]:
### Extracting basic statistics from the dataset
total_messages = df.shape[0]
media_messages = df[df['Message'] == '<Media omitted>'].shape[0]
links = np.sum(df.Url_Count)
print('Group Chatting Stats : ')
print('Total Number of Messages : {}'.format(total_messages))
print('Total Number of Media Messages : {}'.format(media_messages))
print('Total Number of Links : {}'.format(links))

Group Chatting Stats : 
Total Number of Messages : 27694
Total Number of Media Messages : 0
Total Number of Links : 0


In [19]:
#Extracting basic statistics of each user : 
l = df.Author.unique()
for i in range(len(l)):
  ### Filtering out messages of particular user
  req_df = df[df["Author"] == l[i]]
  ### req_df will contain messages of only one particular user
  print(f'--> Stats of {l[i]} <-- ')
  ### shape will print number of rows which indirectly means the number of messages
  print('Total Message Sent : ', req_df.shape[0])
  ### Word_Count contains of total words in one message. Sum of all words/ Total Messages will yield words per message
  words_per_message = (np.sum(req_df['Word\'s']))/req_df.shape[0]
  w_p_m = ("%.3f" % round(words_per_message, 2))  
  print('Average Words per Message : ', w_p_m)
  ### media conists of media messages
  media = sum(req_df["Media_Count"])
  print('Total Media Message Sent : ', media)
  ### links consist of total links
  links = sum(req_df["Url_Count"])   
  print('Total Links Sent : ', links)   
  print()
  print('----------------------------------------------------------n')

--> Stats of Ayesha PUCIT <-- 
Total Message Sent :  1619
Average Words per Message :  5.920
Total Media Message Sent :  0
Total Links Sent :  0

----------------------------------------------------------n
--> Stats of Celal PUCIT <-- 
Total Message Sent :  617
Average Words per Message :  7.170
Total Media Message Sent :  0
Total Links Sent :  0

----------------------------------------------------------n
--> Stats of F.Bilal Uol <-- 
Total Message Sent :  3338
Average Words per Message :  9.190
Total Media Message Sent :  77
Total Links Sent :  0

----------------------------------------------------------n
--> Stats of Umar Ghaffar PUCIT <-- 
Total Message Sent :  1649
Average Words per Message :  7.010
Total Media Message Sent :  0
Total Links Sent :  0

----------------------------------------------------------n
--> Stats of Amna Ali PUCIT <-- 
Total Message Sent :  230
Average Words per Message :  8.260
Total Media Message Sent :  0
Total Links Sent :  0

-------------------------

In [65]:
data = [df['Message']]
rd = pd.concat(data, axis=1)

type(rd)

pandas.core.frame.DataFrame

In [66]:
rd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27694 entries, 0 to 29956
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Message  27694 non-null  object
dtypes: object(1)
memory usage: 432.7+ KB


In [67]:
### new Data Frame
#rd = df['Message']

### Remove all Emoji and other Symbols
rd = rd.replace(r'\W*$', '' , regex=True)

rd = rd.replace(r'^\s*$', np.nan , regex=True)

#rd = df[df['Message'].notna()]
rd = rd[rd['Message'].notna()]

In [68]:
rd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24457 entries, 0 to 29956
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Message  24457 non-null  object
dtypes: object(1)
memory usage: 382.1+ KB


In [69]:
rd

,Message
0,Hmme na pagal ho chuki har teacher ki mintaien...
1,Karain gey
2,"Kbhe sessional, kbhe assignment kbhe presenta..."
3,Jitni marzi karwa lo. Bs quiz ya Viva nhi dena
4,Or phir teachers k krny waly kam b me he kron
...,...
29951,😂😂😂😂 Bus Focus on your study you will get soon
29952,I don't need that type of aurat jo MSCS ki wja...
29953,😂😂😂 Fir to chutti kr
29954,Aho chutti hi ki hoi hai


In [70]:
rd = rd.sample(frac = 1)

In [71]:
len(rd)

mode = [''] * len(rd)

print(mode[:10])

rd['Mode'] = mode

### Rearranging the columns for better understanding
rd = rd[['Mode','Message']]

rd['Words'] = rd['Message'].apply(lambda s : len(s.split(' ')))

['', '', '', '', '', '', '', '', '', '']


In [49]:
rd

,Mode,Message,Words
10061,,Hn g,2
5690,,Han i agree,3
29702,,tu dkh idr kya leny aya,6
13150,,Han lein gy,3
1496,,"🙉🙉🙉🙉 Q? Umar bilal, shakir in ko nai soch skty",10
...,...,...,...
8777,,Yarrr ignore kru please,4
11918,,Parhae se jaan jate h ab tu,7
18567,,Aj to ap aai thi,5
29188,,Ohh ik to ya dukh,5


In [58]:
#rd.to_csv('data.csv', index= False)  ## file size 20k+ ......

In [72]:
rd = rd[rd.Words != 1]
rd = rd[rd.Words != 2]

In [73]:
rd

,Mode,Message,Words
18780,,Pta lgy to btaon gi,5
6084,,Ye wohi hai jisko apne meri lab mein dekha tha...,16
29809,,Ya b acha h,4
24155,,This message was deleted,4
7538,,tu ty chup kr ja,5
...,...,...,...
25953,,Abdullah ko add kr den,5
3231,,Hm bhi isi category me atay hain,7
25342,,Acha chalo koi ni,4
19553,,Na khila kr kaha Jana hai us ny,8


In [74]:
message = rd['Message']

In [81]:
type(message)

pandas.core.series.Series

In [82]:
for i in message:
    print (i)

Pta lgy to btaon gi
Ye wohi hai jisko apne meri lab mein dekha tha jab hum ToC parh rahay they
Ya b acha h
This message was deleted
tu ty chup kr ja
yeh uncle bi tu muft ki pay lete pora saal sirf inhoan ny chand dekhna hota wo bi sai ni dekhty
Apko kia lgta h k already ni bny huye
Ayesha lagta ha ap abi b kafi pechy chal rhi ho
Gwahi b kon dy rha hai
Me to kuch bola e ni bro abi Wo to shukur e aasma lrki e to kuch kaha ni me ne
Hmien ye end tk smhj nh ani
Ye bahir wala kn h or usko kya bt pta h in dono cheezon py depend krti h situation
uncle da pta kro ay kerry pind dy aa kerry shehar dy aa
larky maza krwa diya
Ap krain sahi wali shairi
😂🤣😂🤣Ye v hai
Nhe ap khain roti
Ma ny b vase send ki thi. Shyd double double ho gai ho gi ab
ni hmare sath ye issue ni h bss ap k msgs hi late milty hain
Alhamdulillah... Bai me larka hi thik
Second thing, ky marks ki base p CR bnta. To i think we all know ky ye rule h. But wo jo baat ki wo comfortable hney pe ki.  Ky everyone is completely known to he

Yeh menu ap ni pta
Welcome *Aneesa and Tania
Ham prhty ni hain rona roty hain bs
Wesy e mzaq tha baii
hmm😅 yeh confirm hy k paper hi keh rahe ho
Koi chance ni h baii
Me b dekh ln 🤣🤣kahin mere 70 me s na hn
Wo SE ki thi
Attendance ho bhi gai
me available hun
Shakir aj se inka rasta alag hum se
me ne sari classes li,participation b ki
mera wese kon tha??yad krta hoan
zarort hi ni bai mujy pichly 1 week sy idea hy tum logon ka
Ni baii mujhy rehny ni dety mujhy bna k rkhein sbko seedha kr don
Mardo ya murdo😂😂😂 kindly correct your word
i like adventure but aj ki tarah ka adventure ni
Agr hmari uni open hoti or sports week hota to hmari class ki team ka kya title hota
Baii m ny clear b kr dia hai idhr udhr na niklain ap
Ap log isko bolain k sbka khyal kry
m ny ab kya diya apko??? ap hath do k mere pechy par gai
jb Brey baat kr re ote...to bache chup rehte
Hame to ik assignment ny karvaya ha serf 5
You deleted this message
dont give me hope again
Knock kncok msg del. Kia kissi n
Smjhny ko iss

Phir ma sary rates bta deta q @923008982661
Umar ko add kro phr
Mtlb tenu me kad dwa dost di list to
Wo kon hain ab
Magar dr. murtaza ne to kaha tha k core ni le sakty across the campus
Apko bara pta h
Wo andhi to ni
u know better then me
zarort hi ni ksii munafiq k kehny p m q kron jb wo khush  hy tumy itni takleef q hoti h itna banta hy tu na dekha kr
Ap chahte hen k group ka ek member km ho jae
choose ap logon ny datamining  r iot krna
cover na kro ab
Lg gya pta baii na chahty huye b
Aray .. Abdullah kahan ha
Ap apni jgah thek jeh rhay hn
Ni ap ka to scene alehda e Ap bht nice kaam krta o
Yup... Think so
Okk noted nai hoga aisa.. But ap mt jana
Ha to dfa krp bakio ko
Choice hai apni apni
Akhir m yhe keh skti hu k password bta do m bta dyti hu dkh k
Na lain.... first me second krwa dain
Ma to gin k 5 ghanty sota hu
G ma ny e kaha tha usy kuch
Na chair malangan nu
SS group me send kr do
Mujhe b formula batao ya vala
Haha eda to nashai
Han ye to h kafi zlalt hui thi hmari b
Group hi gr

Bai aj ma serious hu bara. Ni sab pkr k hame zalel kar rhy hn. Ni aur jaga kam hn kya zalel hony vali
Me ap ki support me hun.. tntion na len✌🏻😂 #SupportAbdullah
Ab tm daad bhi ni dy rahi
fawad khan ho tum yr
Yr tb bht busy tha.... Ghr construction ka kam chl ra tha.... Ais liye time nai milta tha
Nhi nhi.. ap ye baat na kahen.. ap ki or un ki sehet me koi zada frq nhi he
Abdullah dakh tery fans wait kr rahe hain kuch aisa bna k dikha dy inko
Yr aik din Eid ka hta us din to lehaz kr lo
ni mzaq theek e....me tbse yehe dekh ra hu Saare yehe soch re umar kb jaye ga or kis ki wja se jaye ga or koi baat e ni he
Meku ab b nai bolna.. Sholi sholi kehty hty
Bai veriables to initialize e ni karvay
Veeray paper dittta kr.. Enna kam ich kuj nai rkheya
You know I have left the other group because of that words
ye grmi kuch kam ho iski wjah sy dil e ni krta bahir niklny ka
Han na knowledge dekho ap bs.. Abhi agay flavors ayein gy.. Mimicry ko choro sasti ya mehngi
Ohh let me check
Ye sticker kro ud

Ye bndi new sticker lai hai
Wohi magar inhn ne papers k kis base pe lagaye hain ye ni samjh a raha
Ye to woi bat ho gai jesy waqar zaka kehta hai k mujhe Prime Minister bnao
Atleast mujhy to
😂😂😂 islamabadion ko to wese hi koi nhi marta.. kuch zada hi masum hote wo.. nhi
Ni apni trf lakin hmain pta to chaly k tra sath bhi koi deta hai
Haha Allah sehat dy isko
Bs sai hai wo boht nice hain
Tm hm ma se nahi ho phir
bs ayesha ko e mra sae naam pta e
Yaar ma tumari video ka wait kr rha hu. Kb a rha ha next clip
Kon c party bai
itni dair tk batein kesy kr lety ho yar
jo direct mac book
Haye... Yeh ha mera veera
Me khud much wala serious hua para aca k result pe
No Aisa koi bat nhe hai
Mjy btana me kick krun ge
Sab sy zyda tu e karta ha usy hurt
Ik to khany ka sun k mujhe b bhook lag jati ha es time
Bai ma kon sa menty kr rha hu k save kry vo
Yeshh she is right on his side
Ye mra official sticker hai
😂😂😂 aho... Hun suspense me mry gi bchari
Kis chz tey
Ap sy e seekha hai asha ki trah shota
lok

tang na kro
Phir kya o ga hmara
or me hardworking b ni hu
Wo pure depression hain... Suna h me ne un ko bhi
Mtlb phur dakh lu
Abdullah shanti achi ni lg ri apko kya
Koi pta ni. Dakho ab
Mzak kr re
Me. Ne ni koi kami mehsoos ni ki Ye galat fehmia na paalein
Chaaa gya mera veera
Just like You
Wo mei nay must layna
2sri power m apko pta
Allah kry hmyn b ho jayn
Yar yehi to msla h bech m chor dety hain kisi side py lgaya krein
pehli dfa tune koi kam ki bat ki
Ohh acha g
Exactly bus shugal lagaty hn us k sath
Sb he pata h
Wese rockstar tu lgda group di peon lag gyi ein
Oh han me pul gya si
Bhai mene abi tk aasma ko aik lfz b nai bola.... Me tw Abdullah ko smjha ra k bt khtm kro
Hum unki baat kr rahay jo illegally lagwa rahay
Aik baar ni do do baar
Aur to Alex bhatti ki bhi boht hai
Start m achi kuch videos thi ab to bongian e marti h
Ye to 5-6 bndo k name hain
Ni tu wo b rehne de
Sai o gya
Yay bat ache tarha samaj arahe h apko
😂😂😂Ap b assume kr rhi kia @923497155520 ki tarah
Ma barish ma na

Hm jese sirf aisi pic e bana skty
Hum to sary bhai bhai hn bro
Uski begum usko khud le gyi hni
Hahah aik sath dono
Jldi krain.... Time bht km hai
Ni ho pana tha hm na bhi to woi kam adhy ghnty ma copy mar liya tha
or tu to beth ja ....hr jaga mu na ghusa lia kr
beta ap side pe rho
Bari baat h ...aik aur week de deya...isse zayda agay ki na sochein
😶😶 hahah terey zayda burey halat hain
Baaiiii wo txt btw full on ignore pr gya he sb se.. jaen rply kr aen
Haha han mjbori h
Warna me kr lena si
Or paging Or Or
Sab se gandi gali soch le apney liye... @923054524120
Ye wesy option achi h saad is m jannat ki guarantee hai
Ab bole koi
Nawaz sharif wala shreef
Ni to me pooch leta apney kisi dost se
smjh gaya bahi
Phr bilal ne uni chor deni thi
phr tre  clients ka kia o ga
Ena to undertaker
This message was deleted
😂😂😂 chand sai parti 🐍 ko
Mr ja ab tu
Unhn ne to ultra Pro max hd surprise dia
Bs yahi jazba chahie tha
Yr iski b sunlo saare
Tm may k ho
Apko yad arhe ha
Nai khair ap ny b kr lia howa..

But ider sb apas me larna shuru ho gaye
Haha hasad kr k knsa 4ka a jana hai
Ye konsi ni bat nikal aai
To phir sy kr lo add
Ab usko smjh aa gai honi k ye kon hai
Isi liye jldi out ho jate hen
MashAllah log purane numbet b yaad rkhte hen.. or hmen kehte hen bht touchy hen ap log numbers k mamle me
shukr hy ap ne bta dia subhu hy....mjhy lga rat k 8:32 hue
😂... Yr odey to bghair mza v ni anda
Phir thek ay
Chal kad fir
Nahe aap krna ...ma dua karun ge. In sha Allah
Saady tu aj kon c dunya ma ghoom rha ha yaar
or 5W's ki sb ki howi thi I guess
bachy ghr m hi achy hoty
Bai group mate ti na
to 2bara add kr dein ge
Isski thori izzat afzai krta hu
Saad 💓 M****b
Han g q k adhe logo ko smj ni ani thi
Mery ko bnna e ni hai
apne office ky bad mere office ajai
Me ne sawal kuch or pocha h
Iqra bat meri or Aimen ki thi bs  Bki sb kyu personal ly rhy
Yar wo hints dy dy ga k kaha se aa skta kesy questions aa skty hain
Kon sa tumara bai
🤣🤣🤣🤣🤣🤣acha bs bs ro q Rae kch ni Kaha mene
Ye sahii mazy hain
Hahaha

Baaaqi sb maaya hai
Mostly sir aisay krtay hn na k students ko reply krna bother nhe krtay....iss k backend pay reason kia ho skti waisay...attitude
eeeeeeeeeeeeeeee😖😖😖😖teri to khair ni ab....mera phone gira dia neechy
🤣🤣🤣Pata chal rha h hameim knsi tareef ho rhi
Me rastey me hi attend kr lwa ga call
Oh sorry ap to nazar e ni aty
@923484608364  tu bewafa niqla yr
🤣🤣🤣 bhai ye zayda ho gya
Or dekhte k kise nzr lgti
Buzurg na name mention kro
Yaar adnan hero lag rha ha vase South Indian movies ka
Bai es ko goli karva tu
Yr to te serious hi le gya
Msla koi ni
Ye btien kahan ja rhe hain
To apna project bta na
Code laga kr
Chal 2sre group me bta di
hahahahahahahah Saad jwab do yr
Me kehti hun soch len
Sarfraz wala question
Celal she kab say huay
Sb maan jaein. ameen
Ap order kren bs hazir Hy sab
Serf double InshaAllah 5 sy 6 gunah ho ga 1 saal ma
Quiz ki kya scene hai.... Kis range main marks aye
Han naa.. too ho skta on campus ho jaen
😍 plan krte bss phr jldi hi
😂😂😂😂 Ashi baat or tu
Shukriy

🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣 auto correct.. Wese pta sch me nhi tha.. wrna star wali correction kr deti
Yaqeeen mano mjy tw apny laptop ka v itna ni pta
Ch sb in ka kuch krien in dono ka
Nai balochistan me hoga
@923497155520 chas a gyi je k nhi
Preshan na ho
Menu sikhaaa ay
Bai wo to already   hai
Ahooo... Me v ohi labn deya si
Bai me teri trha wela ni betha hta har tym
Yaar na keh... Me tujy janta e ni nikal
sir ko aj pay mili hni
Ennu kaddo yar
ho sakta bai wo tenu hands free pe sad songs sun ri ho
Me too correct kr raha tha baat ko srf
Nend ki to baat na e kry. Ya to kahi b a jati ha
Keh kon raha hai jis na sari tiyari ki hoi honi
Jo programming me weak ho wo kese smjhe issy
Mei nhe krti but mei aisaa kaam nhe krti koee mujy baat kr skay
Apko kbi ye sharrf hasil hua
Han han hm esy pesy ikathy kr k non mess dy dyn gy
72kg its enough ithink
Adnan shb ko appreciate krein ap
Yaar ma to adnan ka fan ho gya hu. Kya acting and dashing personality. Yaar tu mujhe apni tictok ka link dy. Mujye follow karna h

In [90]:
import re
reg = '[^\sA-Za-z]'
s = 'Sir bus hamri thori izat rakh rhy hn aur methi methi kr rhy hn'
s= '😂😂😂😂😂😂😂 error to me beech me se nikal lu ga oska fiker na kro'
rs = re.findall(reg,s)

In [92]:
len(rs)

7

In [93]:
def remove_emoji_msg(msg):
    import re
    reg_Exp = '[^\sA-Za-z]'
    len(msg)
    new_msg = []
    for i in msg:
        rs = re.findall(reg_Exp, i)
        if len(rs) == 0:
            new_msg.append(i)
    return new_msg

In [94]:
_list = remove_emoji_msg(message)

In [96]:
_list[:10]

['Pta lgy to btaon gi',
 'Ye wohi hai jisko apne meri lab mein dekha tha jab hum ToC parh rahay they',
 'Ya b acha h',
 'This message was deleted',
 'tu ty chup kr ja',
 'yeh uncle bi tu muft ki pay lete pora saal sirf inhoan ny chand dekhna hota wo bi sai ni dekhty',
 'Apko kia lgta h k already ni bny huye',
 'Ayesha lagta ha ap abi b kafi pechy chal rhi ho',
 'Gwahi b kon dy rha hai',
 'Me to kuch bola e ni bro abi Wo to shukur e aasma lrki e to kuch kaha ni me ne']

In [97]:
len(_list)

14282